In [1]:
import os

In [2]:
list_file = list(os.walk('./'))[0][2]
for file in list_file:
    if '替代' in file:
        alternate = './' + file

print(f'\talternate: \t{alternate}')

	alternate: 	./替代料总表05-29.xlsx


In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=1)[['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6']].fillna('')

df_alternate = df_alternate[df_alternate['替换料1'] != ''].reset_index(drop=True)

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6
0,USCA171876D-3,USCA171876D,USCA171876-3,USCA171876,,,
1,USGS43640B,USGS43640,,,,,
2,USBDA04106,USBDA04106X,,,,,
3,USGS6310TNZB,USGS6310TZ,USGS6310T,USGS6310,USGS6310NZ,USGS6310TNZ,
4,USGS61560B,USGS61560,,,,,
...,...,...,...,...,...,...,...
19340,USDS76734-C,USDS76734,,,,,
19341,USDS76803-C,USDS76803,,,,,
19342,USDS76934-C,USDS76934,,,,,
19343,USDS86429-C,USDS86429,,,,,


In [5]:
def clean_alternate(df_alternate):
    temp_df_alternate = pd.merge(left=df_alternate, right=df_alternate, how='left', left_on='主料SKU', right_on='替换料1', suffixes=('', '_01')).fillna('')
    temp_df_alternate = pd.merge(left=temp_df_alternate, right=df_alternate[df_alternate['替换料2'] != ''], how='left', left_on='主料SKU', right_on='替换料2', suffixes=('', '_02')).fillna('')
    temp_df_alternate = pd.merge(left=temp_df_alternate, right=df_alternate[df_alternate['替换料3'] != ''], how='left', left_on='主料SKU', right_on='替换料3', suffixes=('', '_03')).fillna('')
    temp_df_alternate = pd.merge(left=temp_df_alternate, right=df_alternate[df_alternate['替换料4'] != ''], how='left', left_on='主料SKU', right_on='替换料4', suffixes=('', '_04')).fillna('')
    temp_df_alternate = pd.merge(left=temp_df_alternate, right=df_alternate[df_alternate['替换料5'] != ''], how='left', left_on='主料SKU', right_on='替换料5', suffixes=('', '_05')).fillna('')
    temp_df_alternate = pd.merge(left=temp_df_alternate, right=df_alternate[df_alternate['替换料6'] != ''], how='left', left_on='主料SKU', right_on='替换料6', suffixes=('', '_06')).fillna('')
    
    list_name = ['1', '2', '3', '4', '5', '6']
    for i in range(len(list_name)):
        for j in range(i+1, len(list_name)):
            temp_df_alternate = pd.merge(left=temp_df_alternate, right=df_alternate[df_alternate['替换料'+list_name[j]] != ''], how='left', left_on='替换料'+list_name[i], right_on='替换料'+list_name[j], suffixes=('', '_'+list_name[i]+list_name[j])).fillna('')

    # = = = = = = = = = = = = = = = = = =
    
    num_columns = len(temp_df_alternate.columns)
    for i in range(len(temp_df_alternate)):
        list_row = list(set(temp_df_alternate.loc[i]))
        if '' in list_row:
            list_row.remove('')
            list_row.sort(reverse=True)
        temp_df_alternate.loc[i] = list_row + [''] * (num_columns - len(list_row))

    df_alternate = temp_df_alternate.drop_duplicates(ignore_index=True)
    
    # = = = = = = = = = = = = = = = = = =
    
    df_alternate = df_alternate[['主料SKU',
                                 '替换料1',
                                 '替换料2',
                                 '替换料3',
                                 '替换料4',
                                 '替换料5',
                                 '替换料6']]
    
    # = = = = = = = = = = = = = = = = = =
    
    return df_alternate

In [6]:
length = len(df_alternate)
i = 0
while True:
    print(i)
    df_alternate = clean_alternate(df_alternate)
    if len(df_alternate) == length:
        break
    length = len(df_alternate)
    i += 1

0
1
2


In [7]:
df_alternate.to_excel('./替代ok.xlsx', index=False)